In [53]:
import pandas as pd
import plotly.graph_objects as go

# ===============================================
# 1. MEMUAT DAN MENYIAPKAN DATA
# ===============================================
file_path = 'steam_store_data_2024.csv'
df = pd.read_csv(file_path)

# Pembersihan kolom harga dan diskon
df['price_numeric'] = df['price'].fillna('$0.00').astype(str).str.replace('$', '', regex=False).astype(float)
df['salePercentage_numeric'] = (
    df['salePercentage'].fillna('0%').astype(str)
    .str.replace('%', '', regex=False)
    .str.replace('-', '', regex=False)
    .astype(int)
)

# Urutan review
review_order = [
    'Overwhelmingly Positive', 'Very Positive', 'Mostly Positive',
    'Mixed', 'Mostly Negative', 'Very Negative', 'Overwhelmingly Negative',
    'No user reviews'
]

for col in ['recentReviews', 'allReviews']:
    df[col] = df[col].fillna('No user reviews')
    df[col] = pd.Categorical(df[col], categories=review_order, ordered=True)

df.rename(columns={'allReviews': 'Review_Category'}, inplace=True)

# ===============================================
# 2. DATA UNTUK CHART
# ===============================================
df_reviews_count = df['recentReviews'].value_counts().reindex(review_order, fill_value=0).reset_index()
df_reviews_count.columns = ['Review Category', 'Count']

# ===============================================
# 3. MEMBANGUN FIGURE
# ===============================================
fig = go.Figure()

# --- Scatter Plot ---
scatter_traces = []
for cat in review_order:
    sub_df = df[df['Review_Category'] == cat]
    trace = go.Scatter(
        x=sub_df['price_numeric'],
        y=sub_df['salePercentage_numeric'],
        mode='markers',
        name=cat,
        text=sub_df['title'],
        hovertemplate='<b>%{text}</b><br>Price: $%{x}<br>Discount: %{y}%',
        marker=dict(size=8, opacity=0.7),
        visible=True  # default aktif
    )
    scatter_traces.append(trace)
    fig.add_trace(trace)

# --- Bar Chart ---
bar_trace = go.Bar(
    x=df_reviews_count['Review Category'],
    y=df_reviews_count['Count'],
    name='Jumlah Game',
    marker_color='skyblue',
    visible=False
)
fig.add_trace(bar_trace)

# --- Pie Chart ---
pie_trace = go.Pie(
    labels=df_reviews_count['Review Category'],
    values=df_reviews_count['Count'],
    hole=0.4,
    name='Proporsi Review',
    visible=False
)
fig.add_trace(pie_trace)

# ===============================================
# 4. VISIBILITY SETUP
# ===============================================
n_scatter = len(scatter_traces)
visibility_scatter = [True]*n_scatter + [False, False]
visibility_bar = [False]*n_scatter + [True, False]
visibility_pie = [False]*n_scatter + [False, True]

# ===============================================
# 5. DROPDOWN FILTER SCATTER
# ===============================================
dropdown_buttons = [
    dict(label="All Categories", method="update",
         args=[{"visible": [True]*n_scatter + [False, False]},
               {"title": {"text": "Hubungan Harga dan Diskon (Semua Review)",
                          "x": 0.5, "xanchor": "center"}}])
]

for i, cat in enumerate(review_order):
    visible_state = [False]*n_scatter + [False, False]
    visible_state[i] = True
    dropdown_buttons.append(
        dict(label=cat, method="update",
             args=[{"visible": visible_state},
                   {"title": {"text": f"Hubungan Harga dan Diskon ({cat})",
                              "x": 0.5, "xanchor": "center"}}])
    )

# ===============================================
# 6. TOMBOL NAVIGASI (selalu muncul)
# ===============================================
bottom_buttons = [
    dict(label="Scatter Plot", method="update",
         args=[{"visible": visibility_scatter},
               {"annotations": [
                   # munculkan teks filter dropdown ketika scatter aktif
                   dict(x=0.0, y=1.14, showarrow=False, xref="paper", yref="paper", align="left")
               ],
                "updatemenus[0].visible": True,  # tampilkan dropdown
                "title": {"text": "Hubungan Harga dan Diskon (Scatter Plot)",
                          "x": 0.5, "xanchor": "center"},
                "xaxis": {"title": "Harga ($)", "visible": True, "autorange": True},
                "yaxis": {"title": "Diskon (%)", "visible": True, "autorange": True}}]),
    dict(label="Bar Chart", method="update",
         args=[{"visible": visibility_bar},
               {"annotations": [],
                "updatemenus[0].visible": False,  # sembunyikan dropdown
                "title": {"text": "Distribusi Jumlah Game Berdasarkan Review Terbaru (Bar Chart)",
                          "x": 0.5, "xanchor": "center"},
                "xaxis": {"title": "Kategori Review", "visible": True, "autorange": True},
                "yaxis": {"title": "Jumlah Game", "visible": True, "autorange": True}}]),
    dict(label="Pie Chart", method="update",
         args=[{"visible": visibility_pie},
               {"annotations": [],
                "updatemenus[0].visible": False,  # sembunyikan dropdown
                "title": {"text": "Proporsi Kategori Review Terbaru (Pie Chart)",
                          "x": 0.5, "xanchor": "center"},
                "xaxis": {"visible": False}, "yaxis": {"visible": False}}])
]

# ===============================================
# 7. LAYOUT
# ===============================================
fig.update_layout(
    updatemenus=[
        # Dropdown filter — default aktif (untuk scatter)
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.02, y=1.15,
            xanchor="left",
            yanchor="top",
            visible=True,
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1,
            pad={"r": 10, "t": 10}
        ),
        # Tombol navigasi bawah — selalu aktif
        dict(
            type="buttons",
            direction="right",
            x=0.5, y=-0.18,
            xanchor="center",
            yanchor="top",
            buttons=bottom_buttons,
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1,
            pad={"r": 10, "t": 10}
        )
    ],
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.95,
        xanchor="right",
        x=1.12
    ),
    margin=dict(t=120, r=180, l=60, b=150),
    template='plotly_white',
    title={
        "text": "Steam Store Data Visualization (Multi-View)",
        "x": 0.5,
        "xanchor": "center"
    },
    height=650,
    showlegend=True,
    transition=dict(duration=600, easing="cubic-in-out")
)

# ===============================================
# 8. TAMPILKAN
# ===============================================
fig.show()
